In [1]:
import pandas as pd
import numpy as np
import tables as tb
from tables import (
    Int8Col, Int16Col, Int32Col, Int64Col,
    UInt8Col, UInt16Col, UInt32Col, UInt64Col,
    Float32Col, Float64Col,
    TimeCol, ComplexCol, StringCol, BoolCol, EnumCol
)

from neuropy.core.neurons import NeuronType, Neurons

In [ ]:
pd.DataFrame({'long_laps': [2, 10, 10], 'short_laps': [10, 11, 10], 'num_replays': [20, 100, 500]})

## Neuron Info

# `Quantifiers: զ`
# զSESS := per session
# զEPCS := per epcoch in ['long', 'short', 'all']

['num_pyr', 'num_inter', 'num_BAD', 'num_total'] # զSESS
['num_pf1D', 'num_pf2D'] ## proportion that are place cells, զEPCS


# Each neuron can be uniquely identified by:
['session_context', 'aclus', 'qclus', 'shank_ids', 'cluster_ids']


## CONCLUSION: Sounds like I need a simple collection that stores a table of neuron properties. Probably shouldn't store the derived properties (fragile_linear_neruon_IDX)

# Existing neuropy.core.Neurons object holds mostly spikes
- Neuron info
- Spikes
- Aggregate measures like firing rate

Convenience properties for use in visualizations, etc like
	- aclu_to_neuron_type_map
	- reverse_cellID_index_map
	- 


In [ ]:
# ==================================================================================================================== #
# 2023-08-01 PyTables Test                                                                                             #
# ==================================================================================================================== #

import tables as tb
from tables import (
    Int8Col, Int16Col, Int32Col, Int64Col,
    UInt8Col, UInt16Col, UInt32Col, UInt64Col,
    Float32Col, Float64Col,
    TimeCol, ComplexCol, StringCol, BoolCol, EnumCol
)

# UInt8 255
# UInt16 65535
# UInt32 4,294,967,295



# Custom Column Definition:
# class NeuronTypeCol(Int32Col):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#     def from_enum(self, value):
#         return value.value

#     def to_enum(self, value):
#         return NeuronType(value)


# class NeuronTable(IsDescription):
#     neuron_type = Int32Col()



# Table Definition:
class NeuronTable(tb.IsDescription):
    neuron_type = Int32Col()


with tb.open_file('neurons.h5', mode='w') as h5file:
    group = h5file.create_group("/", 'neurons', 'Neuron data')
    table = h5file.create_table(group, 'table', NeuronTable, "Neuron types")

    # Serialization
    row = table.row
    row['neuron_type'] = NeuronType.PYRAMIDAL.value
    row.append()
    table.flush()

    # Deserialization
    for r in table.iterrows():
        neuron_type = NeuronType(r['neuron_type'])
        print(neuron_type)  # Output will be NeuronType.PYRAMIDAL

In [ ]:
class Particle(tb.IsDescription):
    name      = StringCol(16)   # 16-character String
    idnumber  = Int64Col()      # Signed 64-bit integer
    ADCcount  = UInt16Col()     # Unsigned short integer
    TDCcount  = UInt8Col()      # unsigned byte
    grid_i    = Int32Col()      # 32-bit integer
    grid_j    = Int32Col()      # 32-bit integer
    pressure  = Float32Col()    # float  (single-precision)
    energy    = Float64Col()    # double (double-precision)

In [6]:




# extended_neuron_id

[/c:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/core/neuron_identities.py:111](vscode://file/c:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/core/neuron_identities.py:111)
```python
class NeuronIdentity(SimplePrintable):
    """NeuronIdentity: A multi-facited identifier for a specific neuron/putative cell 
        Used to retain a the identity associated with a value or set of values even after filtering and such.

        cell_uid: (aclu) [2:65]
        shank_index: [1:12]
        cluster_index: [2:28]
        
        ['shank','cluster']
    
    """
    @property
    def extended_identity_tuple(self):
        """The extended_identity_tuple property."""
        return NeuronExtendedIdentityTuple(self.shank_index, self.cluster_index, self.cell_uid) # returns self as a NeuronExtendedIdentityTuple 
    @extended_identity_tuple.setter
    def extended_identity_tuple(self, value):
        assert isinstance(value, NeuronExtendedIdentityTuple), "value should be a NeuronExtendedIdentityTuple"
        self.cell_uid = value.id
        self.shank_index = value.shank
        self.cluster_index = value.cluster
        
    @property
    def extended_id_string(self):
        """The extended_id_string property."""
        return self._extended_id_string
    
    
    
    def __init__(self, cell_uid, shank_index, cluster_index, color=None):
        self.cell_uid = cell_uid
        self.shank_index = shank_index
        self.cluster_index = cluster_index
        self.color = color

    @classmethod
    def init_from_NeuronExtendedIdentityTuple(cls, a_tuple: NeuronExtendedIdentityTuple, a_color=None):
        """Iniitalizes from a NeuronExtendedIdentityTuple and optionally a color
        Args:
            a_tuple (NeuronExtendedIdentityTuple): [description]
            a_color ([type], optional): [description]. Defaults to None.
        """
        return cls(a_tuple.id, a_tuple.shank, a_tuple.cluster, color=a_color)
        
```

[/c:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/core/neuron_identities.py:225](vscode://file/c:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/core/neuron_identities.py:225)
```python
class NeuronIdentitiesDisplayerMixin:
    @property
    def neuron_ids(self):
        """ like self.neuron_ids """
        raise NotImplementedError
    @property
    def neuron_extended_ids(self):
        """ list of NeuronExtendedIdentityTuple named tuples) like tuple_neuron_ids """
        raise NotImplementedError

    @property
    def neuron_shank_ids(self):
        return [extended_id.shank for extended_id in self.neuron_extended_ids]
    
    @property
    def neuron_cluster_ids(self):
        return [extended_id.cluster for extended_id in self.neuron_extended_ids]

    def get_extended_neuron_id_string(self, neuron_i=None, neuron_id=None):
        assert (neuron_i is not None) or (neuron_id is not None), "You must specify either neuron_i (index) or neuron_id, and the other will be returned"
        assert (neuron_i is None) or (neuron_id is None), "You cannot specify both neuron_i (index) and neuron_id, as it would be ambiguous which takes priority. Please remove one of the two arguments."
        
        # TODO: make more general
        curr_cell_alt_id = self.neuron_extended_ids[neuron_i]
        curr_cell_shank = curr_cell_alt_id.shank
        curr_cell_cluster = curr_cell_alt_id.cluster
        return f'(shank {curr_cell_shank}, cluster {curr_cell_cluster})'
        
        # ax1.set_title(
        #     f"Cell {neuron_ids[cell]} - (shank {curr_cell_shank}, cluster {curr_cell_cluster}) \n{round(np.nanmax(pfmap),2)} Hz"
        # )
        
    def other_neuron_id_string(self, neuron_i):
        # TODO: implement as below
        raise NotImplementedError
        # # sorted_neuron_id_labels = [f'Cell[{a_neuron_id}]' for a_neuron_id in sorted_neuron_ids]
        # sorted_neuron_id_labels = [f'C[{sorted_neuron_ids[i]}]({sorted_alt_tuple_neuron_ids[i][0]}|{sorted_alt_tuple_neuron_ids[i][1]})' for i in np.arange(len(sorted_neuron_ids))]

```